# `feyntrop` tutorial: A 2-loop 3-point Feynman graph

The first step is to import the `feyntrop` module. 
Assuming `feyntrop.so` and `py_feyntrop.py` are in the working directory, we can run:

In [1]:
from py_feyntrop import *

## The diagram
* We specify the Feynman diagram via a list of weighted edges with associated masses.
* Each item of the list is given in the format $((u,w),\nu, m^2)$, where $u$ and $w$ are the vertices connected by the edge, $\nu$ is the edge weight, and $m^2$ is the squared mass. 
* We use 0-indexing. So, the vertices are numbered $0,1,2,\ldots$.
* External vertices must come first, so $V_\text{ext} = \{0,1,2\}$ label external partcles, and $V_\text{int} = \{3\}$.

The diagram is then given as an edge list:

In [2]:
edges = [((0,1), 1, 'mm'), ((1,3), 1, 'mm'), ((3,2), 1, 'mm'), ((2,0), 1, 'mm'), ((0,3), 1, 'mm')]

## Kinematics
* Next we specify momentum variables.
* We must provide replacement rules for all scalar products $p_i \cdot p_j \, , \, 0 \leq i \leq j \leq |V_\text{ext}|-2$. 
* In this case $|V_\text{ext}| = 3$, so we have $p_0^2, \, p_1^2$ and $p_0 \cdot p_1$.
* The rules are given by tuples `(sp[i,j], var_str)`, where `sp` stands for 'scalar product' and `var_str` is some string of variables or a number. 

In this example let us immediately set $p_0^2 = p_1^2 = 0$, and then define the single momentum variable $p_2^2$ in terms of the scalar product $p_0 \cdot p_1$ by $p_2^2 = (-p_0 - p_1)^2 = 2 \cdot p_0 \cdot p_1$.

In [3]:
replacement_rules = [(sp[0,0], '0'), (sp[1,1], '0'), (sp[0,1], 'pp2/2')]

* We have two symbolic variables in `edges` and `replacement_rules`, namely `mm` and `pp2`, which must be given numerical values.

We choose $m^2 = 0.2$ and $p_2^2 = 1$:

In [4]:
phase_space_point = [('mm', 0.2), ('pp2', 1)]

One can inspect the $V \times V$ dimensional scalar product matrix $\mathcal{P}^{u,v} = p_u \cdot p_v$ (where $V = |V_\text{ext}| + |V_\text{int}|$) and the list of squared edge masses as follows:

In [5]:
P_uv, m_sqr_list = prepare_kinematic_data(edges, replacement_rules, phase_space_point)
print(P_uv)
print(m_sqr_list)

[[0, 0.500000000000000, -0.500000000000000, 0], [0.500000000000000, 0, -0.500000000000000, 0], [-0.500000000000000, -0.500000000000000, 1.00000000000000, 0], [0, 0, 0, 0]]
[0.2, 0.2, 0.2, 0.2, 0.2]


## Additional settings
* `D0` is the integer part of the spacetime dimension $D = D_0 - 2\epsilon$.
* We expand up to but not including `eps_order`.
* `Lambda` denotes the deformation parameter $\lambda$.
* `N` is the number of Monte Carlo sampling points.

In [6]:
D0 = 2
eps_order = 5
Lambda = 7.6
N = int(1e7)

## Tropical integration
* `trop_res` is the value of the Feynman integral *without* any prefactor.
* `Itr` is the normalization factor in the tropical measure.

In [7]:
trop_res, Itr = tropical_integration(N, D0, Lambda, eps_order, edges, replacement_rules, phase_space_point)

Prefactor: gamma(2*eps + 3).
(Effective) kinematic regime: Minkowski (generic)
Generalized permutahedron property: fulfilled.
Analytic continuation: activated. Lambda = 7.6
Started integrating using 8 threads and N = 1e+07 points.
Finished in 5.83201 seconds = 0.00162 hours.

-- eps^0: [-46.57  +/- 0.13]  +  i * [ 87.21  +/- 0.12]
-- eps^1: [-274.42 +/- 0.55]  +  i * [111.36  +/- 0.55]
-- eps^2: [-435.16 +/- 1.30]  +  i * [-174.27 +/- 1.33]
-- eps^3: [-192.05 +/- 2.15]  +  i * [-494.65 +/- 2.14]
-- eps^4: [218.87  +/- 2.69]  +  i * [-432.23 +/- 2.67]


This yields the $\epsilon$-expansion *with* the prefactor $\Gamma(\omega)/\Gamma(\nu_1) \cdots \Gamma(\nu_E) = \Gamma(2\epsilon+3)$ included, where $\omega$ is the superficial degree of divergence and $E$ is the number of edges.

In [8]:
expansion = eps_expansion(trop_res, edges, D0)
print(expansion)

174.4129387*I - 93.13287765 + eps*(-720.7142072 + 544.6098867*I) + eps**2*(-2115.404931 + 497.3087663*I) + eps**3*(-3572.746352 - 676.1642402*I) + eps**4*(-3874.524586 - 2725.868268*I) + O(eps**5)
